## Import Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
from pandas.core.common import SettingWithCopyWarning

# Suppress SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


## Import,  Clean, and Aggregate Data

In [2]:
# CONSTANTS

# Find agency ID of Cedar City Police Department
agencies = pd.read_csv("UT-2022/agencies.csv", dtype=str)
agency_id_list = agencies.loc[agencies['county_name'] == 'IRON', 'agency_id'].tolist()

fire_season = [5, 6, 7, 8, 9, 10]

agency_id_list

['19922', '19923', '19925', '19926', '19927', '33456']

### Data Processing Pipeline

The following functions make up a data processing pipeline for the data. Since the original data is stored and reported yearly, I will need to read each CSV file and combine them together.

The first read_data() function reads the data file and modify the column names to make sure the data combination in the following process is smooth.

The second filter_incident() function filters the incidents that happened around Cedar City area. I also filter the month to only select those incidents happened in fire season.

The third aggregate_offense() function combines the "incident" to its corresponding offense type.

In [3]:
def read_data(year):
    # Import the data needed to compute count data
    # Define the common folder path
    folder_path = f"UT-{year}/"
    
    # Read 
    offense_type = pd.read_csv(folder_path + "nibrs_offense_type.csv", dtype=str)
    offense_type.columns = [x.lower() for x in offense_type.columns]
    
    offense_year = pd.read_csv(folder_path + "nibrs_offense.csv", dtype=str)
    offense_year.columns = [x.lower() for x in offense_year.columns]
    
    incident_year = pd.read_csv(folder_path + "nibrs_incident.csv", dtype=str)
    incident_year.columns = [x.lower() for x in incident_year.columns]
    
    return offense_type, offense_year, incident_year
    
def filter_incident(year, agency_id_list, incident_year, offense_year):
    # Filter incidents processed by PD around Cedar City
    incident_cedar = incident_year[incident_year['agency_id'].isin(agency_id_list)]

    # Filter incidents happened in fire season
    incident_cedar['incident_date'] = pd.to_datetime(incident_cedar['incident_date'])
    incident_cedar['data_month'] = incident_cedar['incident_date'].dt.month
    incident_cedar = incident_cedar[incident_cedar['data_month'].isin(fire_season)]    
    if 'data_year' not in incident_cedar.columns:
        incident_cedar['data_year'] = incident_cedar['incident_date'].dt.year

    # Find all offense corresponding to incidents ID filtered above
    offense_cedar = offense_year[offense_year['incident_id'].isin(incident_cedar.incident_id)]
    if 'data_year' not in offense_cedar.columns:
        offense_cedar['data_year'] = year
    return offense_cedar

def aggregate_offense(offense_cedar, offense_type):
    # Combine offense and type together, aggregate to get count
    if "offense_type_id" in offense_cedar.columns:
        df_year = pd.merge(offense_cedar, offense_type, how="left", on="offense_type_id")
    else:
        df_year = pd.merge(offense_cedar, offense_type, how="left", on="offense_code")
    df_year = df_year.groupby(['data_year', 'offense_code', 'offense_name']).count()[['offense_id']].reset_index()
    df_year.rename(columns={'offense_id': 'offense_count'}, inplace=True)
    
    return df_year


This is a function combining all 3 functions defined above to see a clear workflow.

In [4]:
def yearly_offense_count(year):
    offense_type, offense_year, incident_year = read_data(year)
    offense_cedar = filter_incident(year, agency_id_list, incident_year, offense_year)
    
    return aggregate_offense(offense_cedar, offense_type)

In [5]:
dfs = []
# crime_df = pd.DataFrame(columns=['data_year', 'offense_code', 'offense_name', 'offense_id'])

for year in range(1993, 2023):
    print(f"Getting aggregate data from {year}...")
    yearly_df = yearly_offense_count(year)
    dfs.append(yearly_df)

crime_df = pd.concat(dfs, ignore_index=True)
crime_df

Getting aggregate data from 1993...
Getting aggregate data from 1994...
Getting aggregate data from 1995...
Getting aggregate data from 1996...
Getting aggregate data from 1997...
Getting aggregate data from 1998...
Getting aggregate data from 1999...
Getting aggregate data from 2000...
Getting aggregate data from 2001...
Getting aggregate data from 2002...
Getting aggregate data from 2003...
Getting aggregate data from 2004...
Getting aggregate data from 2005...
Getting aggregate data from 2006...
Getting aggregate data from 2007...
Getting aggregate data from 2008...
Getting aggregate data from 2009...
Getting aggregate data from 2010...
Getting aggregate data from 2011...
Getting aggregate data from 2012...
Getting aggregate data from 2013...
Getting aggregate data from 2014...
Getting aggregate data from 2015...
Getting aggregate data from 2016...
Getting aggregate data from 2017...
Getting aggregate data from 2018...
Getting aggregate data from 2019...
Getting aggregate data from 

,data_year,offense_code,offense_name,offense_count
0,2000,11A,Rape,2
1,2000,220,Burglary/Breaking & Entering,3
2,2000,23D,Theft From Building,8
3,2000,23E,Theft From Coin-Operated Machine or Device,1
4,2000,23F,Theft From Motor Vehicle,2
...,...,...,...,...
245,2022,35A,Drug/Narcotic Violations,207
246,2022,35B,Drug Equipment Violations,144
247,2022,370,Pornography/Obscene Material,8
248,2022,520,Weapon Law Violations,33


In [6]:
crime_df.to_csv("nibrs_crime_data.csv", index=False)